Notebook References:

- xarray access: https://github.com/EOPF-Sample-Service/eopf-sample-notebooks/blob/main/notebooks/Sentinel-2/Sentinel-2_L1C_MSI_Zarr_product_exploration.ipynb
- xcube-eopf: https://eopf-sample-service.github.io/eopf-sample-notebooks/introduction-xcube-eopf-plugin
- xcube-stac: https://github.com/xcube-dev/xcube-stac/blob/main/examples/notebooks/cdse_sentinel_2.ipynb

`conda install xcube-stac xcube-eopf`


In [1]:
import cartopy.crs as ccrs
import numpy as np
#import matplotlib.pyplot as plt
import xarray as xr
#import xarray_eopf
import requests

import dask
from xcube.core.store import new_data_store, get_data_store_params_schema
from xcube_eopf.utils import reproject_bbox
#import xcube
#import xcube_eopf
#import xcube_stac

#for direct loading
import rioxarray
import fsspec
import s3fs

# for benchmarking
from dataclasses import dataclass
from typing import List
from itertools import product
import pandas as pd
import time

## 1. Read directly from link

In [2]:
# hamburg notebook returns:

# eopf:
# https://stac.browser.user.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2A_MSIL2A_20250503T103701_N0511_R008_T32UNE_20250503T173316?.language=de
# https://stac.browser.user.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2C_MSIL2A_20250501T104041_N0511_R008_T32UNE_20250501T161558?.language=de
# https://stac.browser.user.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2B_MSIL2A_20250506T103629_N0511_R008_T32UNE_20250506T115207?.language=de

# https://stac.core.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2A_MSIL2A_20250503T103701_N0511_R008_T32UNE_20250503T173316
# https://stac.core.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2C_MSIL2A_20250501T104041_N0511_R008_T32UNE_20250501T161558
# https://stac.core.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2B_MSIL2A_20250506T103629_N0511_R008_T32UNE_20250506T115207

# cdse equivalents: 
# https://browser.stac.dataspace.copernicus.eu/collections/sentinel-2-l2a/items/S2A_MSIL2A_20250503T103701_N0511_R008_T32UNE_20250503T173316?.language=de
# https://browser.stac.dataspace.copernicus.eu/collections/sentinel-2-l2a/items/S2C_MSIL2A_20250501T104041_N0511_R008_T32UNE_20250501T161558?.language=de
# https://browser.stac.dataspace.copernicus.eu/collections/sentinel-2-l2a/items/S2B_MSIL2A_20250506T103629_N0511_R008_T32UNE_20250506T115207?.language=de

# https://stac.dataspace.copernicus.eu/v1/collections/sentinel-2-l2a/items/S2A_MSIL2A_20250503T103701_N0511_R008_T32UNE_20250503T173316
# https://stac.dataspace.copernicus.eu/v1/collections/sentinel-2-l2a/items/S2C_MSIL2A_20250501T104041_N0511_R008_T32UNE_20250501T161558
# https://stac.dataspace.copernicus.eu/v1/collections/sentinel-2-l2a/items/S2B_MSIL2A_20250506T103629_N0511_R008_T32UNE_20250506T115207

In [3]:
# todo: is it possible to access a file directly via xcube?
#path_eopf_zarr = "https://objectstore.eodc.eu:2222/e05ab01a9d56408d82ac32d69a5aae2a:sample-data/tutorial_data/cpm_v253/S2B_MSIL1C_20250113T103309_N0511_R108_T32TLQ_20250113T122458.zarr"
path_eodc_zarr = "https://objectstore.eodc.eu:2222/e05ab01a9d56408d82ac32d69a5aae2a:202505-s02msil2a/03/products/cpm_v256/S2A_MSIL2A_20250503T103701_N0511_R008_T32UNE_20250503T173316.zarr"
path_eodc_safe = "https://objects.eodc.eu/e05ab01a9d56408d82ac32d69a5aae2a:notebook-data/SAFE/S2A_MSIL2A_20250503T103701_N0511_R008_T32UNE_20250503T173316.SAFE"
path_cdse_safe = "s3://eodata/Sentinel-2/MSI/L2A/2025/05/03/S2A_MSIL2A_20250503T103701_N0511_R008_T32UNE_20250503T173316.SAFE"

### EOPF Zarr on EODC

In [4]:
dt = xr.open_datatree(path_eodc_zarr, engine="zarr", chunks={})

/tmp/ipykernel_6104/3000600587.py:1: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  dt = xr.open_datatree(path_eodc_zarr, engine="zarr", chunks={})
/tmp/ipykernel_6104/3000600587.py:1: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta6

In [5]:
%%time
band_eodc_zarr = dt["measurements/reflectance/r10m"]["b04"].load()

CPU times: user 2.74 s, sys: 1.17 s, total: 3.92 s
Wall time: 2.12 s


In [6]:
band_eodc_zarr

<xarray.DataArray 'b04' (y: 10980, x: 10980)> Size: 964MB
array([[0.0818, 0.0915, 0.0966, ...,    nan,    nan,    nan],
       [0.0739, 0.1026, 0.1208, ...,    nan,    nan,    nan],
       [0.069 , 0.1102, 0.1338, ...,    nan,    nan,    nan],
       ...,
       [0.2148, 0.2218, 0.2404, ...,    nan,    nan,    nan],
       [0.1872, 0.2032, 0.2386, ...,    nan,    nan,    nan],
       [0.1754, 0.1932, 0.2158, ...,    nan,    nan,    nan]])
Coordinates:
  * x        (x) int64 88kB 499985 499995 500005 500015 ... 609755 609765 609775
  * y        (y) int64 88kB 5999995 5999985 5999975 ... 5890225 5890215 5890205
Attributes:
    _eopf_attrs:     {'add_offset': -0.1, 'coordinates': ['x', 'y'], 'dimensi...
    dtype:           <u2
    fill_value:      0
    long_name:       BOA reflectance from MSI acquisition at spectral band b0...
    proj:bbox:       [499980.0, 5890200.0, 609780.0, 6000000.0]
    proj:epsg:       32632
    proj:shape:      [10980, 10980]
    proj:transform:  [10.0, 0.0, 499980.0, 0.0, -10.0, 6000000.0, 0.0, 0.0, 1.0]
    proj:wkt2:       PROJCS["WGS 84 / UTM zone 32N",GEOGCS["WGS 84",DATUM["WG...
    units:           digital_counts
    valid_max:       65535
    valid_min:       1

### EOPF SAFE on EODC

In [7]:
%%time
# Full URL to the B04 10m band file, from f"{path_eopf_zarr}/manifest.safe"
b04_url = (
    f"{path_eodc_safe}/GRANULE/L2A_T32UNE_A051514_20250503T103937/IMG_DATA/R10m/"
    "T32UNE_20250503T103701_B04_10m.jp2"
)

# Open and read the band
fs = fsspec.filesystem("http")
with fs.open(b04_url) as f:
    band_eodc_safe = rioxarray.open_rasterio(f, masked=True)

band_eodc_safe.name = "B04"
band_eodc_safe

CPU times: user 165 ms, sys: 86.8 ms, total: 252 ms
Wall time: 348 ms


<xarray.DataArray 'B04' (band: 1, y: 10980, x: 10980)> Size: 482MB
[120560400 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 88kB 5e+05 5e+05 5e+05 ... 6.098e+05 6.098e+05
  * y            (y) float64 88kB 6e+06 6e+06 6e+06 ... 5.89e+06 5.89e+06
    spatial_ref  int64 8B 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

### SAFE on CDSE S3

In [8]:
# cdse credentials
credentials = {
    "key": "FTE4ZT820RDZTHOU6I8C",
    "secret": "EdSaK2k1DjJm1rTlbucDaaSsmSSawWFz9da9Wemz",
}

In [9]:
%%time

# Your CDSE credentials
fs = s3fs.S3FileSystem(
    key=credentials["key"],
    secret=credentials["secret"],
    client_kwargs={
        "region_name": "eu-central-1",
        "endpoint_url": "https://s3.dataspace.copernicus.eu"
    }
)

# Correct path from manifest.safe
band_path = (
    "eodata/Sentinel-2/MSI/L2A/2025/05/03/"
    "S2A_MSIL2A_20250503T103701_N0511_R008_T32UNE_20250503T173316.SAFE/"
    "GRANULE/L2A_T32UNE_A051514_20250503T103937/IMG_DATA/R10m/"
    "T32UNE_20250503T103701_B04_10m.jp2"
)

# Open the file from S3
with fs.open(band_path, mode="rb") as f:
    band = rioxarray.open_rasterio(f, masked=True)

band.name = "B04"
band

CPU times: user 644 ms, sys: 115 ms, total: 759 ms
Wall time: 2.15 s


<xarray.DataArray 'B04' (band: 1, y: 10980, x: 10980)> Size: 482MB
[120560400 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 88kB 5e+05 5e+05 5e+05 ... 6.098e+05 6.098e+05
  * y            (y) float64 88kB 6e+06 6e+06 6e+06 ... 5.89e+06 5.89e+06
    spatial_ref  int64 8B 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

## 2. Read via querry
The influence of the underlying STAC API (EOPF vs CDSE) and the implementation of the used software (xcube-stac vs xcube-eopf) affects the performance.

### Functions

In [33]:
def create_aoi(bbox, reduction):
    """
    Generate a reduced bounding box or centroid based on a reduction factor.
    Helper function to easily create portions of the original bbox around the centroid.

    Parameters:
    - bbox: [min_lon, min_lat, max_lon, max_lat]
    - reduction: float between 0 and 1
        - 0 returns the centroid as (lon, lat)
        - 0 < reduction < 1 returns a scaled bounding box centered at the centroid

    Returns:
    - reduced bounding box list
    """
    if not (0 <= reduction <= 1):
        raise ValueError("Reduction must be between 0 and 1.")

    min_lon, min_lat, max_lon, max_lat = bbox

    # Compute centroid
    centroid_lon = (min_lon + max_lon) / 2
    centroid_lat = (min_lat + max_lat) / 2

    #if reduction == 0:
     #   return (centroid_lon, centroid_lat)

    # Compute reduced bounding box dimensions
    lat_span = (max_lat - min_lat) * reduction
    lon_span = (max_lon - min_lon) * reduction

    return [
        centroid_lon - lon_span / 2,
        centroid_lat - lat_span / 2,
        centroid_lon + lon_span / 2,
        centroid_lat + lat_span / 2,
    ]

In [34]:
# this is used for defining inputs 
@dataclass
class BenchmarkConfig:
    data_id: str
    bbox: List[float]
    time_range: List[str]
    spatial_res: int
    crs: str
    variables: List[str] 


In [35]:
# access function eopf
def access_eopf(cfg: BenchmarkConfig):
    return store_zarr.open_data(
        data_id=cfg.data_id,
        bbox=cfg.bbox,
        time_range=cfg.time_range,
        spatial_res=cfg.spatial_res,
        crs=cfg.crs,
        variables=cfg.variables,
    ).load()


In [36]:
# access function safe
def access_safe(cfg: BenchmarkConfig):
    return store_safe.open_data(
        data_id=cfg.data_id,
        bbox=cfg.bbox,
        time_range=cfg.time_range,
        spatial_res=cfg.spatial_res,
        crs=cfg.crs,
        asset_names=upper(cfg.variables),
    ).load()


In [37]:
# benchmark function
# loops through the given configs
def benchmark_data_access(configs, access_fn):
    results = []
    
    for cfg in configs:
        bbox_repr = reproject_bbox(cfg.bbox, "EPSG:4326", cfg.crs) # has to be done for xcube
        cfg.bbox = bbox_repr # TODO: THIS IS REPLACING THE VALUE IN THE CONFIG OBJECT ALSO OUTSIDE OF THE FUNCTION!!!!!!
        print(cfg.bbox)
        print(f"Running: {cfg}")
        start = time.perf_counter()
        ds = access_fn(cfg)
        end = time.perf_counter()

        results.append({
            "data_id": cfg.data_id[0],
            "bbox": bbox_repr,
            "time_range": cfg.time_range,
            "spat_res": cfg.spatial_res,
            "crs": cfg.crs,
            "variables": cfg.variables,
            "duration_sec": round(end - start, 4)
        })
        
    return pd.DataFrame(results)


### Parameter Definitions

In [38]:
# pull the bbox from the catalog/object here
url = "https://stac.core.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2A_MSIL2A_20250503T103701_N0511_R008_T32UNE_20250503T173316"
response = requests.get(url)
item = response.json()
bbox = item["bbox"]
print(bbox)

[8.99969379936479, 53.15557577629945, 10.371024273615161, 54.148104103961266]


In [39]:
# pull native crs
crs_native = item['properties']['proj:code'] # "EPSG:32632"
print(crs_native)

EPSG:32632


In [40]:
# define data id
opt_data_id = [
    "sentinel-2-l2a"
]

In [41]:
# define bboxes
# only in lat/lon, reprojection of bbox to chosen crs happens later in code
opt_bbox = [
    #create_aoi(bbox, 0), # pixel
    create_aoi(bbox, 256 / 10980), # ml patch approx 256*256
    #create_aoi(bbox, 0.125), # eight
    create_aoi(bbox, 0.25), # quarter
    bbox, # full  
]

In [42]:
# define crs
# mandatory in xcube
# if it differs from native crs processing is enforced (reprojection, resampling)
opt_crs = [
    crs_native, 
    "EPSG:4326",
    #"EPSG:3035",
]

In [43]:
# define times
opt_time_range = [
    ["2025-05-01", "2025-06-01"], # day
    ["2025-05-01", "2025-05-07"], # month
    ["2024-01-01", "2025-01-01"] # year
]

In [44]:
# define spatial resolution
# everything deviating from native resolution enforces processing (resampling)
opt_spatial_res = [
    10, 
    #20, 
    100,
]

In [45]:
# define band combinations
# choosing bands with different resolutions enforces processing (resampling)
opt_variables = [
    ["b02"],
    ["b02", "b04"],
    #[]
]


In [46]:
# create all combinations of parameters specified above
configs = [
    BenchmarkConfig(data_id, bbox, time_range, spatial_res, crs, variables)
    for data_id, bbox, time_range, spatial_res, crs, variables in product(
        opt_data_id,
        opt_bbox,
        opt_time_range,
        opt_spatial_res,
        opt_crs,
        opt_variables
    )
]
print(f"Number of configs: {len(configs)}")
print(configs[0])

Number of configs: 72
BenchmarkConfig(data_id='sentinel-2-l2a', bbox=[9.669372670305636, 53.64026948239441, 9.701345402674315, 53.66341039786631], time_range=['2025-05-01', '2025-06-01'], spatial_res=10, crs='EPSG:32632', variables=['b02'])


In [47]:
# Create custom dataclass object
custom_cfg = BenchmarkConfig(
    data_id=opt_data_id[0],
    bbox=opt_bbox[0],
    time_range=opt_time_range[0],
    spatial_res=opt_spatial_res[0],
    crs=opt_crs[0],
    variables=opt_variables[0]
)
print(custom_cfg)

BenchmarkConfig(data_id='sentinel-2-l2a', bbox=[9.669372670305636, 53.64026948239441, 9.701345402674315, 53.66341039786631], time_range=['2025-05-01', '2025-06-01'], spatial_res=10, crs='EPSG:32632', variables=['b02'])


## STAC EOPF

In [48]:
store_zarr = new_data_store("eopf-zarr")

In [49]:
# store_zarr.list_data_ids()
# store_zarr.get_open_data_params_schema(data_id="sentinel-2-l2a")

In [50]:
df_benchm_eopf = benchmark_data_access(configs[0:2], access_eopf)

(544229.9589484755, 5943707.259587298, 546367.9651581453, 5946302.084300316)
Running: BenchmarkConfig(data_id='sentinel-2-l2a', bbox=(544229.9589484755, 5943707.259587298, 546367.9651581453, 5946302.084300316), time_range=['2025-05-01', '2025-06-01'], spatial_res=10, crs='EPSG:32632', variables=['b02'])
(544229.9589484755, 5943707.259587298, 546367.9651581453, 5946302.084300316)
Running: BenchmarkConfig(data_id='sentinel-2-l2a', bbox=(544229.9589484755, 5943707.259587298, 546367.9651581453, 5946302.084300316), time_range=['2025-05-01', '2025-06-01'], spatial_res=10, crs='EPSG:32632', variables=['b02', 'b04'])


In [51]:
configs[0:2] # bbox is altered globally by function benchmark_data_access!!!!!!!!

[BenchmarkConfig(data_id='sentinel-2-l2a', bbox=(544229.9589484755, 5943707.259587298, 546367.9651581453, 5946302.084300316), time_range=['2025-05-01', '2025-06-01'], spatial_res=10, crs='EPSG:32632', variables=['b02']),
 BenchmarkConfig(data_id='sentinel-2-l2a', bbox=(544229.9589484755, 5943707.259587298, 546367.9651581453, 5946302.084300316), time_range=['2025-05-01', '2025-06-01'], spatial_res=10, crs='EPSG:32632', variables=['b02', 'b04'])]

In [52]:
df_benchm_eopf # data_id is iterated - bbox is altered - add number of pixels/sqkm count
# IS THERE CACHING GOING ON, TWO BANDS QUICKER THAN ONE? HOW TO DISABLE CACHING?

,data_id,bbox,time_range,spat_res,crs,variables,duration_sec
0,s,"(544229.9589484755, 5943707.259587298, 546367....","[2025-05-01, 2025-06-01]",10,EPSG:32632,[b02],13.7471
1,s,"(544229.9589484755, 5943707.259587298, 546367....","[2025-05-01, 2025-06-01]",10,EPSG:32632,"[b02, b04]",9.1805


In [30]:
%%time

bbox_repr = reproject_bbox(custom_cfg.bbox, "EPSG:4326", custom_cfg.crs)
print(bbox_repr)

ds_zarr = store_zarr.open_data(
    data_id=custom_cfg.data_id,
    bbox=bbox_repr,
    time_range=custom_cfg.time_range,
    spatial_res=custom_cfg.spatial_res,
    crs=custom_cfg.crs,
    variables=custom_cfg.variables,
)
ds_zarr.load()

(533869.2479094835, 5931106.512426242, 556794.2915226527, 5958929.887704585)
CPU times: user 8.29 s, sys: 648 ms, total: 8.93 s
Wall time: 15.1 s


<xarray.Dataset> Size: 664MB
Dimensions:      (time: 13, y: 2783, x: 2294)
Coordinates:
  * time         (time) datetime64[ns] 104B 2025-05-01T10:40:41.025000 ... 20...
    spatial_ref  int64 8B 0
  * x            (x) float64 18kB 5.339e+05 5.339e+05 ... 5.568e+05 5.568e+05
  * y            (y) float64 22kB 5.959e+06 5.959e+06 ... 5.931e+06 5.931e+06
Data variables:
    b02          (time, y, x) float64 664MB nan 0.0306 0.0307 ... nan nan nan
Attributes: (4)

## STAC CDSE

In [31]:
credentials = {
    "key": "xxx",
    "secret": "xxx",
}

In [28]:
# get_data_store_params_schema("stac-cdse")

In [33]:
store_safe = new_data_store("stac-cdse", stack_mode=True, **credentials)

In [34]:
# bands have to be renamed
bands_cdse = ['B02']

In [ ]:
%%time
ds_safe = store_safe.open_data(
    data_id=custom_cfg.data_id[0],
    bbox=bbox_repr,
    time_range=custom_cfg.time_range,
    spatial_res=custom_cfg.spatial_res,
    crs=custom_cfg.crs,
    asset_names=bands_cdse,
)
ds_safe.load()